In [1]:
import csv
import pandas as pd
import numpy as np
import os

In [2]:
items = ['apple', 'pen']
item_short_name = {'ap': 'apple', 'pe': 'pen'}

# Read sell information

In [3]:
def get_sell_info(filename):
    raw_sell_info = pd.read_csv(filename)
    states = raw_sell_info['sku_num'].map(lambda x: x.split('-')[0])
    state = states[0]
    raw_sell_info['sku_num'] = raw_sell_info['sku_num'].map(lambda x: x.split('-')[2])
    
    for sh_i, item in item_short_name.items():
        raw_sell_info[item] = raw_sell_info['sku_num'].map(lambda x: int(x == sh_i))
    
    
    sell_info = pd.DataFrame(columns=['date'] + items)
    
    grouped = raw_sell_info.groupby('date')
    for date, gr in grouped:
        ser = pd.Series(index = ['date'] + items)
        ser[items] = gr[items].sum()
        ser['date'] = date
        sell_info = sell_info.append(ser, ignore_index=True)
    
    sell_info[items] = sell_info[items].astype(int)
    
    sell_info = sell_info.set_index('date')
    return sell_info, state

In [4]:
if os.path.isfile('temporary/sell_info.csv'):
    sell_info = pd.read_csv('temporary/sell_info.csv', index_col='date')
    with open('temporary/state') as file:
        state = file.readline()
else:
    sell_info, state = get_sell_info('inputdata/sell.csv')
    sell_info.to_csv('temporary/sell_info.csv')
    with open('temporary/state', 'w') as file:
        file.write(state)

# Read inventory and supply information

In [ ]:
inventory_info = pd.read_csv('inputdata/inventory.csv', index_col='date')
supply_info = pd.read_csv('inputdata/supply.csv', index_col='date')

# add date from inventory and supply to sell (if it wasn't there before)

In [ ]:
def add_row(df_from, df_to):
    was_changes = False
    for index in df_from.index:
        if index not in df_to.index:
            was_changes = True
            df_to.loc[index] = pd.Series(index = items, data = 0)
    
    return was_changes

In [ ]:
def add_sel(sell_info, inventory_info, supply_info):
    wc = add_row(inventory_info, sell_info)
    wc = wc or add_row(supply_info, sell_info)
    if wc:
        sell_info = sell_info.sort_index()
        
    return sell_info

In [ ]:
sell_info = add_sel(sell_info, inventory_info, supply_info)

# calculate number of items in store for every day

In [ ]:
def get_daily(sell_info, supply_info, inventory_info):    
    last_remain = pd.Series(index=items, data=0)

    store_info = pd.DataFrame(columns= ['date'] + items)
    stolen_info = pd.DataFrame(columns=['date'] + items)

    for date, row in sell_info.iterrows():
        if date in supply_info.index:
            sup = supply_info.loc[date]
        else:
            sup = pd.Series(index=items, data=0)

        should_be = last_remain + sup - row

        if date in inventory_info.index:        
            cur_inv = inventory_info.loc[date]
            add_store = cur_store = cur_inv
            add_store['date'] = date
            store_info = store_info.append(add_store, ignore_index=True)
            
            cur_st = should_be - cur_inv
            cur_st['date'] = date
            stolen_info = stolen_info.append(cur_st, ignore_index=True)
        else:        
            cur_store = should_be
            should_be['date'] = date
            store_info = store_info.append(should_be, ignore_index=True)
        last_remain = cur_store
        
    store_info[items] = store_info[items].astype(int)
    stolen_info[items] = stolen_info[items].astype(int)
    return store_info, stolen_info

In [ ]:
if os.path.isfile('result/daily.csv'):
    daily = pd.read_csv('result/daily.csv')
    stolen_info = pd.read_csv('temporary/stolen.csv')
else:
    daily, stolen_info = get_daily(sell_info, supply_info, inventory_info)
    daily.to_csv('result/daily.csv', index=False)
    stolen_info.to_csv('temporary/stolen.csv', index=False)

In [ ]:
def get_annual_report(df):
    df['year'] = df['date'].map(lambda x: x.split('-')[0])
    annual_report = pd.DataFrame()
    grouped = df.groupby('year')
    
    for year, gr in grouped:
        ser = pd.Series(index=items)
        ser[items] = gr[items].sum()
        ser['year'] = year
        annual_report = annual_report.append(ser, ignore_index=True)
    annual_report[items] = annual_report[items].astype(int)
    return annual_report

In [ ]:
sell_info = sell_info.reset_index()

In [ ]:
stolen_annual = get_annual_report(stolen_info)
sold_annual = get_annual_report(sell_info)

In [ ]:
full_annual = pd.merge(stolen_annual, sold_annual, how='outer', suffixes=['_stolen', '_sold'], on='year')
full_annual = full_annual.fillna(0)
full_annual['state'] = state

In [ ]:
columns_order = ['year', 'state']
for item in items:
    columns_order += [item + '_sold', item + '_stolen']
full_annual.to_csv('result/annual_reports.csv', index=False, columns = columns_order)